# DuckDB Kernel Demo

In [1]:
import duckdb
from deltalake import write_deltalake

In [2]:
con = duckdb.connect()

## create Delta table

In [3]:
df1 = con.query(
    "SELECT i AS id, i % 2 AS part, 'value-' || i AS value FROM range(0, 5) tbl(i)"
).df()

In [4]:
print(df1)

   id  part    value
0   0     0  value-0
1   1     1  value-1
2   2     0  value-2
3   3     1  value-3
4   4     0  value-4


In [6]:
write_deltalake(f"./fun_delta_table", df1)

In [8]:
duckdb.sql("SELECT * FROM delta_scan('./fun_delta_table') ORDER BY id;").show()

┌───────┬───────┬─────────┐
│  id   │ part  │  value  │
│ int64 │ int64 │ varchar │
├───────┼───────┼─────────┤
│     0 │     0 │ value-0 │
│     1 │     1 │ value-1 │
│     2 │     0 │ value-2 │
│     3 │     1 │ value-3 │
│     4 │     0 │ value-4 │
└───────┴───────┴─────────┘



## append to Delta table

In [9]:
df2 = con.query(
    "SELECT i AS id, i % 2 AS part, 'value-' || i AS value FROM range(5, 10) tbl(i)"
).df()

In [12]:
write_deltalake(f"./fun_delta_table", df2, mode="append")

In [13]:
duckdb.sql("SELECT * FROM delta_scan('./fun_delta_table') ORDER BY id;").show()

┌───────┬───────┬─────────┐
│  id   │ part  │  value  │
│ int64 │ int64 │ varchar │
├───────┼───────┼─────────┤
│     0 │     0 │ value-0 │
│     1 │     1 │ value-1 │
│     2 │     0 │ value-2 │
│     3 │     1 │ value-3 │
│     4 │     0 │ value-4 │
│     5 │     1 │ value-5 │
│     6 │     0 │ value-6 │
│     7 │     1 │ value-7 │
│     8 │     0 │ value-8 │
│     9 │     1 │ value-9 │
├───────┴───────┴─────────┤
│ 10 rows       3 columns │
└─────────────────────────┘



## Read table with deletion vectors

In [15]:
path = "/Users/matthew.powers/data/out/reader_tests/generated/deletion_vectors/delta"

In [20]:
duckdb.sql(f"SELECT * FROM delta_scan('{path}');").show()

┌─────────┬───────┬────────────┐
│ letter  │  int  │    date    │
│ varchar │ int64 │    date    │
├─────────┼───────┼────────────┤
│ b       │   228 │ 1978-12-01 │
└─────────┴───────┴────────────┘

